# data processing into a single dataframe


In [5]:
import pandas as pd
'''
for i in 2 3 4; do python3 keep_relevant_headers.py \
../data/chr2-4_with_gene/chr$i\_gene.tsv \
../data/chr2-4_with_gene/chr$i.out_set1.tsv \
--cols chr,pos,ref,alt,aapos,APPRIS,TSL,codonpos,Ancestral_allele,ExAC_cnv.score,\
HUVEC_fitCons_score,clinvar_clnsig,Interpro_domain,GDI,LoFtool_score,SORVA_LOF_MAF0.005_HetOrHom,\
Essential_gene_CRISPR; done

'''


df_all = pd.read_csv('../chr2_out.tsv', sep='\t', encoding = "UTF-8")
df_all = df_all.set_index(['chr','pos', 'ref', 'alt']) #index by identifier

for c in range (3,4):
    path = '../chr' + str(c) + '_out.tsv'
    df = pd.read_csv(path, sep='\t', encoding = "UTF-8")
    df = df.set_index(['chr','pos', 'ref', 'alt']) #index by identifier
    df_all= df_all.append(df)
    
print('concat shape', df_all.shape)

df = df_all
print("shape before", df.shape)
df = df.replace({'.': 0})  ## FIXME: atm replace with nan and drop: later find a way to impute, maybe 0 but not sure
df = df.dropna()
print("shape after dropping all data with score = '.'", df.shape)

## still contains some mixed labels, see them by: df[~df['clinvar_clnsig'].isin(list(clinvar_labels.keys()))]

features_mapping = {
    'APPRIS': {
        'principal1': 5,
        'principal2': 4, 
        'principal3': 3, 
        'principal4': 2, 
        'principal5': 1,
        'alternative1': -1,
        'alternative2': -2
    },
    'Ancestral_allele': {
        'ACTG': 2,
        'actg': 1,
        'N': 1,
        '-': -2,
    },
    'Essential_gene_CRISPR': {
        'E': -1,
        'N': 1
    }
}

## Map labels to 1/0 and remove Unkn
clinvar_labels={
    'Pathogenic': 1,
    'Likely_pathogenic': 1,
    'Pathogenic/Likely_pathogenic': 1,
    'Benign/Likely_benign': 0,
    'Likely_benign': 0,
    'Benign': 0
}
df = df.replace(features_mapping)

df = df[df['clinvar_clnsig'].isin(list(clinvar_labels.keys()))]  # best to drop everything that isn't in the dict
df['Problematic'] = df['clinvar_clnsig'].map(clinvar_labels)

df= df.drop(columns=['clinvar_clnsig', 'Interpro_domain'])#, 'clinvar_id'])  #tbh, I don't know why I kept them. consider adding to index

print("shape after dropping Unk", df.shape)

df.head()
### pickle to reuse dataframe:
df.to_pickle("./pickled_dfset1_chr234.pkl")

FileNotFoundError: [Errno 2] No such file or directory: '../chr2_out.tsv'